In [3]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# instalments = pd.read_csv("../../../../homecredit/installments_payments.csv")
application_train = pd.read_csv('../../../../homecredit/application_train.csv')
# pos_cash= pd.read_csv("../../../../homecredit/POS_CASH_balance.csv")
# previous_application = pd.read_csv("../../../../homecredit/previous_application.csv")
# credit_card_balance= pd.read_csv("../../../../homecredit/credit_card_balance.csv")
# bureau_balance= pd.read_csv("../../../../homecredit/bureau_balance.csv")
# bureau= pd.read_csv("../../../../homecredit/bureau.csv")

# Train

In [5]:
# List of columns to drop
columns_to_drop = [
    'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE','FLAG_EMAIL',
    'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
    'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
    'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG',
    'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
    'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG',
    'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
    'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
    'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
    'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI',
    'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
    'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
    'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE',
    'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE',
    'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
    'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
    'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
    'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK'
]

application_train= application_train.drop(columns=columns_to_drop)

In [6]:
application_train['365243DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].apply(lambda x: 1 if x == 365243 else 0)
application_train['DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].replace(365243, np.nan)

In [7]:
# change DAYS_BIRTH to AGE
application_train["DAYS_BIRTH"] = application_train["DAYS_BIRTH"] / 365
application_train["DAYS_BIRTH"] = application_train["DAYS_BIRTH"].astype(int).abs()
application_train = application_train.rename(columns={"DAYS_BIRTH": "AGE"})

# change DAYS_EMPLOYED to JOB_TENURE
application_train["DAYS_EMPLOYED"] = application_train["DAYS_EMPLOYED"] / 365
application_train["DAYS_EMPLOYED"] = application_train["DAYS_EMPLOYED"].round(1).abs()
application_train = application_train.rename(columns={"DAYS_EMPLOYED": "JOB_TENURE"})

#處理極端值
application_train['AMT_INCOME_TOTAL'] = application_train['AMT_INCOME_TOTAL'].replace(100000000, np.nan)
application_train[[
    'DAYS_REGISTRATION','DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE']] = application_train[[
    'DAYS_REGISTRATION','DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE']].abs()


application_train['REGION_RATING_CLIENT_W_CITY'] = application_train['REGION_RATING_CLIENT_W_CITY'].replace(-1, 0)


# (11) Replace values greater than 100 in 'AMT_REQ_CREDIT_BUREAU_QRT' column with np.nan
application_train['AMT_REQ_CREDIT_BUREAU_QRT'] = application_train['AMT_REQ_CREDIT_BUREAU_QRT'].apply(lambda x: np.nan if x > 100 else x)

#缺失值補0
application_train[[
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR','OWN_CAR_AGE','EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3','AMT_REQ_CREDIT_BUREAU_QRT','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']] = application_train[[
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR','OWN_CAR_AGE','EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3','AMT_REQ_CREDIT_BUREAU_QRT','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']].fillna(0)

#缺失值補平均數
columns_to_fill_mean = ['AMT_ANNUITY','AMT_GOODS_PRICE',"JOB_TENURE",'DAYS_LAST_PHONE_CHANGE','CNT_FAM_MEMBERS']  
mean_values = {}
for col in columns_to_fill_mean:
    mean_values[col] = application_train[col].mean()
    application_train.loc[application_train[col].isna(), col] = mean_values[col]


#缺失值補隨機值
columns_to_fill_random = ['NAME_TYPE_SUITE', 'OCCUPATION_TYPE']

for col in columns_to_fill_random:
    random_values = np.random.choice(application_train[col].dropna(), size=application_train[col].isna().sum())
    application_train.loc[application_train[col].isna(), col] = random_values


In [8]:
print(application_train.isna().sum()/application_train.shape[0])
application_train.shape

SK_ID_CURR                     0.0
TARGET                         0.0
NAME_CONTRACT_TYPE             0.0
CODE_GENDER                    0.0
FLAG_OWN_CAR                   0.0
FLAG_OWN_REALTY                0.0
CNT_CHILDREN                   0.0
AMT_INCOME_TOTAL               0.0
AMT_CREDIT                     0.0
AMT_ANNUITY                    0.0
AMT_GOODS_PRICE                0.0
NAME_TYPE_SUITE                0.0
NAME_INCOME_TYPE               0.0
NAME_EDUCATION_TYPE            0.0
NAME_FAMILY_STATUS             0.0
NAME_HOUSING_TYPE              0.0
REGION_POPULATION_RELATIVE     0.0
AGE                            0.0
JOB_TENURE                     0.0
DAYS_REGISTRATION              0.0
DAYS_ID_PUBLISH                0.0
OWN_CAR_AGE                    0.0
OCCUPATION_TYPE                0.0
CNT_FAM_MEMBERS                0.0
REGION_RATING_CLIENT           0.0
REGION_RATING_CLIENT_W_CITY    0.0
WEEKDAY_APPR_PROCESS_START     0.0
HOUR_APPR_PROCESS_START        0.0
ORGANIZATION_TYPE   

(307511, 39)

In [10]:
application_train.to_csv('D:/learn/PLOTY/Homecredit/application_train.csv', index=False)